### This notebook includes codes for LA-Discretization algorithms developed in Mandal et al. (2024) paper. 
### Please note that modules and functions are listed in util_LA.py script

#### First we import the required packages and modules from util_baseline.py script

In [1]:
import pandas as pd
import copy
from util_LA import travel_time_computer, generate_E_star, create_buckets, create_neighborhoods, generate_P_plus, creating_efficient_frontier, construct_R, construct_capacity_graph, construct_time_graph
from util_LA import create_E_u_star_dict, calculate_a_wvr, calculate_E_star_uw_dict, calculate_aw_star_r, mip_solver, lp_relaxation_solver
from util_LA import merge_buckets, compute_k, expand_capacity_buckets, expand_time_buckets

#### Data for different experiments is imported from excel file. We also add another row for the ending node. The ending with its attributes are added to the last row of df

In [2]:
df = pd.read_csv('R102.csv')
df = df.iloc[:51]
# Adding ending node depot
new_row = df.iloc[0].copy() 
new_row["CUST NO."] = len(df) + 1
new_row_df = pd.DataFrame([new_row])
df = pd.concat([df, new_row_df], ignore_index=True)

#### Preprocessing data

In [3]:
# extracting data of customers and depots from the dataframe
node_data = {}
for idx, row in df.iterrows():
    cid = int(row['CUST NO.'])  # or keep as float if you prefer
    node_data[cid] = {
        'x':        float(row['XCOORD.']),
        'y':        float(row['YCOORD.']),
        'demand':   float(row['DEMAND']),
        'early':    float(row['READY TIME']),
        'late':      float(row['DUE DATE']),
        'service':  float(row['SERVICE TIME'])
    }

# Vehicle maximum time and capacity capacity
d0 = 200.0
t0 = node_data[1]["late"] # this is based on Solomon dataset. The total time is equal to uppper bound of depot time window
k = 15

# Getting time windows based on bedget remaining logic
for u in node_data:
    node_data[u]["t_plus_budget"] = t0 - node_data[u]["early"]  # Adjusted t^+ for budget logic
    node_data[u]["t_minus_budget"] = t0 - node_data[u]["late"]  # Adjusted t^- for budget logic


# Indices for depots
alpha = 1
bar_alpha = len(node_data)  

# Extracting all customers from all nodes
all_nodes = list(node_data.keys())  
customers = [n for n in all_nodes if n not in (alpha, bar_alpha)]

customer_demands = {n: node_data[n]['demand'] for n in customers}


In [4]:
E_star = generate_E_star(all_nodes, node_data, alpha, bar_alpha, d0)

# Create the buckets dictionary
ds = 5   # used as increment factor for creating capacity buckets
ts = 50  # used as increment factor for creating time buckets
Du = {}  # initalizing capacity buckets
Tu = {}  # initalizing time buckets
for u in customers:
    Du[u] = create_buckets(customer_demands[u], ds, d0)
    Tu[u] = create_buckets(node_data[u]["t_minus_budget"], ts, node_data[u]["t_plus_budget"])

#### The input parameters are initialized

In [5]:
# inital parameters
k = 6
iter_max = 10
min_inc = 1
zeta = 9

##### Here eficient frontier is created

In [6]:
neighborhoods = create_neighborhoods(node_data, alpha+1, bar_alpha-1, k, t0, d0)

P_plus = generate_P_plus(neighborhoods, node_data)


R_dict = creating_efficient_frontier(node_data, P_plus)

R = {}
for u in customers:
    R[u] = construct_R(u, R_dict, neighborhoods[u], bar_alpha, max_subset_size=5)

#### LA-discretization algorithm

In [7]:
iter_since_reset = 0
last_lp_val = -9999999
N_u = neighborhoods

In [8]:
while iter_since_reset < iter_max:
    prev_N_u = copy.deepcopy(N_u)
    prev_Tu = copy.deepcopy(Tu)
    prev_Du = copy.deepcopy(Du)
    # Step 5-7: Reset neighborhoods if iter_since_reset >= zeta
    if iter_since_reset >= zeta:
        for u in customers:
            N_u [u] = neighborhoods[u]

    # Step 8: Solve LP relaxation
    # zD, zT, lp_objective_value = lp_relaxation_solver(all_nodes, customers, customer_demands, node_data, E_star, t0, d0, alpha, bar_alpha, R, Du, Tu, N_u)
    capacity_duals, time_duals, pi_uwvk, pi_uwk, zD, zT, lp_objective_value = lp_relaxation_solver(all_nodes, customers, customer_demands, node_data, E_star, t0, d0, alpha, bar_alpha, R, Du, Tu, N_u)

    # Step 9-15: Check for LP improvement and apply contraction
    if lp_objective_value > last_lp_val + min_inc:
        # Line 10-12: Apply contraction operations
        for u in customers:
   
            Du[u] = merge_buckets(u, Du[u], capacity_duals)
            Tu[u] = merge_buckets(u, Tu[u], time_duals)
            k_u = compute_k(u, N_u[u], pi_uwk, pi_uwvk, E_star)
            N_u[u] = N_u[u][:k_u] if k_u < len(N_u[u]) else N_u[u]
        # breakpoint()            
            # print(f'capacity bucket is : {Du[u]}')
            # print(f'time bucket is : {Tu[u]}')
        # Update LP objective and iteration count
        last_lp_val = lp_objective_value
        iter_since_reset = 0
    # Step 16-17: Expand buckets for sufficiency
    Du = expand_capacity_buckets(Du, zD, customer_demands, d0, alpha, bar_alpha)
    Tu = expand_time_buckets(Tu, zT, travel_time_computer, node_data, t0, alpha, bar_alpha)
    # print('**********************************************************************')
    # print(f'capacity bucket is : {Du}')
    # print(f'time bucket is : {Tu}')
    # Increment iteration count
    iter_since_reset += 1
    # N_u == prev_N_u and Tu == prev_Tu and Du == prev_Du
    if N_u == prev_N_u and Tu == prev_Tu and Du == prev_Du:
        break
    print(last_lp_val)

Set parameter Username
Set parameter LicenseID to value 2602720
Academic license - for non-commercial use only - expires 2025-12-22
852.3220338679753
904.6647389522201
931.001042550078
934.2124000000006
934.2124000000006
935.5924000000005
935.5924000000005
935.5924000000005
935.5924000000005
935.5924000000005
935.5924000000005
935.5924000000005
935.5924000000005
935.5924000000005
935.5924000000005


#### Now, the mixed integer linear programming model is solved using the adjusted Du, Tu, and N_u

In [9]:
for u in customers:
    Du[u] = merge_buckets(u, Du[u], capacity_duals)
    Tu[u] = merge_buckets(u, Tu[u], time_duals)
    k_u = compute_k(u, N_u[u], pi_uwk, pi_uwvk, E_star)
    N_u[u] = N_u[u][:k_u] if k_u < len(N_u[u]) else N_u[u]


In [10]:
mip_solver(all_nodes, customers, customer_demands, node_data, E_star, t0, d0, alpha, bar_alpha, R, Du, Tu, N_u)

Set parameter TimeLimit to value 1200
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[x86] - Darwin 22.1.0 22A400)

CPU model: Intel(R) Core(TM) i5-1038NG7 CPU @ 2.00GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Non-default parameters:
TimeLimit  1200

Optimize a model with 6277 rows, 46760 columns and 65075 nonzeros
Model fingerprint: 0xd06bf3af
Variable types: 45328 continuous, 1432 integer (1432 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+02]
  Objective range  [5e+00, 8e+01]
  Bounds range     [1e+00, 2e+02]
  RHS range        [1e+00, 2e+02]
Presolve removed 2319 rows and 34728 columns
Presolve time: 0.40s
Presolved: 3958 rows, 12032 columns, 84675 nonzeros
Variable types: 10579 continuous, 1453 integer (1434 binary)
Found heuristic solution: objective 1870.9500000

Root relaxation: objective 9.114300e+02, 3123 iterations, 0.17 seconds (0.23 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     